In [4]:
import requests
import json
import pandas as pd
import datetime as dt
import pprint
import pycountry
import time

import pandas_datareader.data as web
from IPython.display import clear_output

Australia, Israel, Japan, Korea, UK, USA
Estonia, France, Germany, Ireland, Italy, Netherlands, Poland, Portugal, Spain, Sweden

c = """AUS, ISR, JPN, KOR, GBR, USA, EST, FRA, DEU, IRL, ITA, NLD, POL, PRT, ESP, SWE"""

In [5]:
pp = pprint.PrettyPrinter()

In [16]:
C_API = {'AUS': '0:15:1:0',
'ISR': '0:13:0:0',
'JPN': '0:13:2:0',
'KOR': '0:13:2:0',
'GBR': '0:10:2:0',
'EST': '0:11:2:0',
'FRA': '0:13:0:0',
'DEU': '0:13:2:0',
'IRL': '0:13:0:0',
'ITA': '0:13:2:0',
'NLD': '0:13:2:0',
'POL': '0:13:2:0',
'PRT': '0:13:2:0',
'ESP': '0:13:2:0',
'SWE': '0:13:2:0'}

start_year = 2008; end_year = dt.datetime.now().year

base = f'''https://stats.oecd.org/SDMX-JSON/data/ALFS_EMP/COUNTRY_CODE.\
YAREV4+YA994TL1_ST+YA994AL1_ST+YA99A4L1_ST+YA994IL1_ST+YA99B4L1_ST+YA99C4L1_ST+YA99D4L1_ST+YA99E4L1_ST+YA99F4L1_ST+YA994SL1_ST+YA99G4L1_ST+YA99H4L1_ST+YA99I4L1_ST+YA99J4L1_ST+YA99K4L1_ST+YA99L4L1_ST+YA99M4L1_ST+YA99N4L1_ST+YA99O4L1_ST+YA99P4L1_ST+YA99Q4L1_ST+YA99X4L1_ST+YA99S4L1_ST+YA99T4L1_ST+YA99U4L1_ST.MA+FE+TT.A/\
all?startTime={start_year}&endTime={end_year}'''

In [20]:
C_CODE = {}
for c in C_API.keys():
    C_CODE[c] = pycountry.countries.get(alpha_3=c).name

print(C_CODE)

{'AUS': 'Australia', 'ISR': 'Israel', 'JPN': 'Japan', 'KOR': 'Korea, Republic of', 'GBR': 'United Kingdom', 'EST': 'Estonia', 'FRA': 'France', 'DEU': 'Germany', 'IRL': 'Ireland', 'ITA': 'Italy', 'NLD': 'Netherlands', 'POL': 'Poland', 'PRT': 'Portugal', 'ESP': 'Spain', 'SWE': 'Sweden'}


In [21]:
print(pycountry.countries.get(alpha_3='AUS').name)
print(C_CODE['AUS'])

Australia
Australia


In [22]:
column_names = ["Country Code", "Country"]
YEARS = [f'{year}' for year in range(start_year, end_year + 1)]
column_names.extend(YEARS)

print(YEARS)
print(column_names)

['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
['Country Code', 'Country', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']


In [26]:
df = pd.DataFrame(columns=column_names)
df.head()

base_row = {column_name:"na" for column_name in column_names}
print(base_row)

{'Country Code': 'na', 'Country': 'na', '2008': 'na', '2009': 'na', '2010': 'na', '2011': 'na', '2012': 'na', '2013': 'na', '2014': 'na', '2015': 'na', '2016': 'na', '2017': 'na', '2018': 'na', '2019': 'na', '2020': 'na', '2021': 'na', '2022': 'na', '2023': 'na'}


In [27]:
for c_code in C_API.keys():
    clear_output(); new_row = base_row.copy()
    new_row["Country Code"] = c_code; new_row["Country"] = C_CODE[c_code]

    link = base.replace("COUNTRY_CODE", c_code)
    req = requests.get(link)

    JS = json.loads(req.text)

    test = JS['dataSets'][0]['series']
    data = test[C_API[c_code]]['observations']
    
    years_dct = JS['structure']['dimensions']['observation'][0]['values']
    years = [year['id'] for year in years_dct]

    o_keys = [data[key][0] for key in data]
    for index in range(len(years)):
        real_year = years[index]
        o_index = o_keys[index]
        
        new_row[real_year] = o_index

    df.loc[len(df)] = new_row

df.head()

,Country Code,Country,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AUS,Australia,341.333,355.298,351.719,362.925,390.442,367.5520,381.6110,398.718,401.4020,428.7460,456.6850,na,na,na,na,na
1,ISR,Israel,na,na,na,na,na,165.9400,174.0000,180.090,185.8200,186.1200,199.9455,222.1281,233.5287,246.7509,na,na
2,JPN,Japan,1890.0,1930.0,1960.0,1850.0,1880.0,1920.0000,2030.0000,2090.000,2070.0000,2130.0000,2200.0000,2290.0,2400.0,2560.0,na,na
3,KOR,"Korea, Republic of",636.3,657.5,670.4,706.2,706.6,696.8333,718.3333,773.500,783.5833,782.8333,837.3333,860.4167,846.6667,900.5833,na,na
4,GBR,United Kingdom,1039.0,1021.0,1016.25,1069.0,1120.25,1174.5000,1202.7500,1263.750,1263.7500,1294.5000,1306.0000,1392.75,1521.5,1580.75,na,na


In [28]:
df

,Country Code,Country,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AUS,Australia,341.333,355.298,351.719,362.925,390.442,367.5520,381.6110,398.718,401.4020,428.7460,456.6850,na,na,na,na,na
1,ISR,Israel,na,na,na,na,na,165.9400,174.0000,180.090,185.8200,186.1200,199.9455,222.1281,233.5287,246.7509,na,na
2,JPN,Japan,1890.0,1930.0,1960.0,1850.0,1880.0,1920.0000,2030.0000,2090.000,2070.0000,2130.0000,2200.0000,2290.0,2400.0,2560.0,na,na
3,KOR,"Korea, Republic of",636.3,657.5,670.4,706.2,706.6,696.8333,718.3333,773.500,783.5833,782.8333,837.3333,860.4167,846.6667,900.5833,na,na
4,GBR,United Kingdom,1039.0,1021.0,1016.25,1069.0,1120.25,1174.5000,1202.7500,1263.750,1263.7500,1294.5000,1306.0000,1392.75,1521.5,1580.75,na,na
5,EST,Estonia,15.65,14.7,12.725,16.725,18.475,19.7000,22.0750,24.375,27.3000,28.8250,28.1750,30.7,29.65,31.875,na,na
6,FRA,France,670.05,720.875,736.275,752.5,744.975,701.0000,720.7500,747.150,742.5000,792.7250,826.5500,874.8,934.375,982.625,na,na
7,DEU,Germany,1229.625,1224.55,1211.95,1237.675,1280.375,1152.9000,1164.2000,1199.225,1259.5750,1268.1750,1324.1250,1369.05,na,1579.975,na,na
8,IRL,Ireland,86.825,88.075,87.95,89.225,93.925,95.7500,97.9500,101.700,108.2000,115.4750,117.6250,123.575,132.825,148.725,na,na
9,ITA,Italy,541.525,536.6,522.575,544.4,554.725,549.0000,551.3750,561.450,562.0500,564.6750,602.8500,618.125,623.725,655.675,na,na


In [14]:
print(df.to_json())

{"Country Code":{"0":"AUS","1":"ISR","2":"JPN","3":"KOR","4":"GBR","5":"EST","6":"FRA","7":"DEU","8":"IRL","9":"ITA","10":"NLD","11":"POL","12":"PRT","13":"ESP","14":"SWE"},"Country":{"0":"Australia","1":"Israel","2":"Japan","3":"Korea, Republic of","4":"United Kingdom","5":"Estonia","6":"France","7":"Germany","8":"Ireland","9":"Italy","10":"Netherlands","11":"Poland","12":"Portugal","13":"Spain","14":"Sweden"},"2008":{"0":341.333,"1":"na","2":1890.0,"3":636.3,"4":1039.0,"5":15.65,"6":670.05,"7":1229.625,"8":86.825,"9":541.525,"10":308.675,"11":294.5,"12":90.95,"13":575.175,"14":181.85},"2009":{"0":355.298,"1":"na","2":1930.0,"3":657.5,"4":1021.0,"5":14.7,"6":720.875,"7":1224.55,"8":88.075,"9":536.6,"10":299.05,"11":318.225,"12":90.55,"13":533.3,"14":179.5},"2010":{"0":351.719,"1":"na","2":1960.0,"3":670.4,"4":1016.25,"5":12.725,"6":736.275,"7":1211.95,"8":87.95,"9":522.575,"10":293.725,"11":302.8,"12":103.45,"13":527.7,"14":174.95},"2011":{"0":362.925,"1":"na","2":1850.0,"3":706.2,"4"

In [8]:
df.to_csv('OECD.csv', index=False, float_format='%.2f')

In [ ]:
c_code = "ISR"
x = base_row.copy()
x["Country Code"] = c_code; x["Country"] = pycountry.countries.get(alpha_3=c_code).name

link = base.replace("COUNTRY_CODE", c_code)
req = requests.get(link)

js = json.loads(req.text)

test = JS['dataSets'][0]['series']
data = test[C_API[c_code]]['observations']

years_dct = JS['structure']['dimensions']['observation'][0]['values']
years = [year['id'] for year in years_dct]

o_keys = [data[key][0] for key in data]
for index in range(len(years)):
    real_year = years[index]
    o_index = o_keys[index]
    
    x[real_year] = o_index

df.loc[len(df)] = x

pp.pprint(data)
pp.pprint(years)
print(x)
# print(c_code)
# data = test[C_CODE[c_code]]['observations']
df.head()

In [ ]:
base = """https://stats.oecd.org/restsdmx/sdmx.ashx/GetDataStructure/ALFS_EMP"""
# <dataset identifier>/<filter expression>/<agency name>[ ?<additional parameters>]"""
r = requests.get(base)

js = json.loads(r.text)
dct = js["Data"]
connectionCode = js["StatusCode"]

match connectionCode:
    case 200:
        print("200: Successful Operation")
    case 400:
        print("400: Missing ResourceID")
    case 404:
        print("404: Resource not found")
    case other:
        print(connectionCode, type(connectionCode))

data_rows = dct["row"]
# pp.pprint(data_rows)

all_dfs = []
for i in range(len(data_rows)):
    df = pd.DataFrame(data_rows[i]['columns'])
    df['Label'] = data_rows[i]['rowText']
    all_dfs.append(df)

big_df = pd.concat(all_dfs)
print(big_df.head())
print()
print(big_df.tail(30))

# big_df.to_csv('singstat.csv', index=False)